## Model Validation

In [ ]:
from pathlib import Path
import time
import pandas as pd
import geopandas as gpd
import numpy as np
import pickle
import networkx as nx
from stochopy.optimize import minimize
from tqdm import tqdm
import similaritymeasures
import random
import matplotlib.pyplot as plt
from shapely.ops import Point, MultiLineString, LineString
from importlib import reload
import datetime
from scipy.spatial.distance import directed_hausdorff
np.set_printoptions(suppress=True)

from bikewaysim.paths import config, stadia_toner, maptiler_streets
from bikewaysim.impedance_calibration import stochastic_optimization, speedfactor
from bikewaysim.network import modeling_turns
from bikewaysim.routing import rustworkx_routing_funcs
from bikewaysim.impedance_calibration import optimization_viz, utils


from step_1_calibration_experiments import all_calibrations, full_model

## Create K-Fold Groups

In [ ]:
with (config['calibration_fp'] / 'subsets.pkl').open('rb') as fh:
    subsets = pickle.load(fh)
trips = [tripids for name, tripids in subsets if name == 'random'][0]

In [ ]:
import numpy as np
from sklearn.model_selection import KFold

kf = KFold(n_splits=5,shuffle=True,random_state=420)
training_folds = []
testing_folds = []

for i, (train_index, test_index) in enumerate(kf.split(trips)):

    train = [trips[idx] for idx in train_index]
    test = [trips[idx] for idx in test_index]

    print(f"Fold {i}:")

    print(f"  Train: len={len(train_index)} index={train}")

    print(f"  Test:  len={len(test_index)} index={test}")

    training_folds.append((f'fold_{i}',train))
    testing_folds.append((f'fold_{i}',test))

    
if (config['calibration_fp']/'validation').exists() == False:
    (config['calibration_fp']/'validation').mkdir()

with (config['calibration_fp']/'validation/training_folds.pkl').open('wb') as fh:
    pickle.dump(training_folds,fh)
with (config['calibration_fp']/'validation/testing_folds.pkl').open('wb') as fh:
    pickle.dump(testing_folds,fh)

## Create bootstrap samples

In [ ]:
import random

num_to_resample = 1000
bootstrap_samples = [(f'bootsample_{idx}',random.choices(trips,k=len(trips))) for idx in range(0,1000)]

In [ ]:
# ~ 420 unique trips in the bootstrapped sample
# inspect how many duplicate trips are in the bootstrap
dups = [len(set(x)) for _, x in bootstrap_samples]
np.array(dups).mean()

In [ ]:
# with (config['calibration_fp']/'validation/bootstrap_samples.pkl').open('wb') as fh:
#     pickle.dump(bootstrap_samples,fh)